# **Lab 3:** Approximate solution of equations by Horner and Sturm methods

## 1. Horner's Method

Implement Horner's Method so that, given a polynomial $P(x)$ of
any degree and a point $x_0 \in \mathbb{R}$, determine another polynomial $Q(x)$ such that

$$ P(x) = (x-x_0) \: Q(x) + P(x_0) $$

In [1]:
import numpy as np

In [2]:
def Horner(P, x_0):
    b = np.empty(len(P.coeffs)-1) #coeficientes de Q vacios con una posicion menos que P
    b[0] = P.coeffs[0] #añado primera posicion
    for i in range(1, len(P.coeffs)-1): #añado el resto (al ser un for->[1...len-2])
        b[i] = P.coeffs[i] + b[i-1] * x_0
    return np.poly1d(b)

In [3]:
# para comprobar
P = np.poly1d([5,7,2,3])
x_0 = [0,3]
for x0 in x_0:
    Q = Horner(P, x0)
    print(np.polyadd(np.polymul(np.array([1, - x0]), Q), np.polyval(P, x0)) == P)

True
True


Use it with the polynomial:

$ P(x) = 5x^3+7x^2+2x+3$ con $x_0=0$ y $x_0 = 3$

## 2. Sturm's Method

1. Implement Sturm's Method to, given a polynomial $P(x)$, **construct the sequence of Sturm polynomials**. Make a function that returns a list of polynomials (ndarrays).

In [2]:
def SturmSequence(P):
    seq = []
    seq.append(P)
    dividendo = P
    i = -1
    while dividendo.order > 1:
        if i == -1:
            divisor = dividendo.deriv()
            seq.append(divisor)
        else:
            q, r = np.polydiv(dividendo, divisor)
            seq.append(-r)  
            dividendo=divisor
            divisor=-r 
        i+=1
    return seq

In [4]:
P = np.poly1d([1,-9,24,-36])
seq = SturmSequence(P)
seq

[poly1d([  1,  -9,  24, -36]),
 poly1d([  3, -18,  24]),
 poly1d([ 2., 12.]),
 poly1d([-240.])]

2. Complement the implementation of section 1, to, using Sturm's Method, **find the number of positive and negative roots** of $P(x)$ in a given interval. Make a function that returns the values `pos` and `neg`.

NOTE: Assume that all roots are simple.

In [6]:
from sympy import Integer

def numberOfRoots(P,a,b): 
    seq = SturmSequence(P) 
    roots = np.zeros(3) #número de cambios de signo en x=a, x=0 y x=b
    intervalo=[a,0,b]
    for i in range(len(intervalo)):
        signo_actual = 'None'
        signo_anterior = 'None'
        for j in range(len(seq)):
            valor = np.polyval(seq[j], intervalo[i])
            if valor > 0:
                signo_actual = 'positivo'
                if signo_anterior == 'None':
                    signo_anterior = signo_actual                   
            elif valor < 0:
                signo_actual = 'negativo'
                if signo_anterior == 'None':
                    signo_anterior = signo_actual
            if signo_actual != signo_anterior:
                roots[i] += 1
                signo_anterior = signo_actual
    return tuple(roots)

In [7]:
P = np.poly1d([1,-9,24,-36])
neg, x0, pos = numberOfRoots(P,-100,100)
(neg, x0, pos)

(2.0, 2.0, 1.0)

3. Use the implementation of section 2 to **separate the roots of a given polynomial** (i.e., find intervals containing one and only one
polynomial** (i.e. to find intervals containing one and only one root). Make a function that returns a list of intervals.

NOTE: Assume that all the roots of the polynomial are real and simple.

In [8]:
def setRootsNegative(P,numeroRaices,intervals):
    i=0
    a=0
    while i<numeroRaices:
        neg=pos=0
        while neg==pos:
            a-=1
            neg,x0,pos = numberOfRoots(P,a,a+1)
        intervals.append([a+1,a])
        i+=1
    return intervals
def setRootsPositive(P,numeroRaices,intervals):
    i=0
    b=0
    while i<numeroRaices:
        neg=pos=0
        while neg==pos:
            b+=1
            neg,x0,pos = numberOfRoots(P,b-1,b)
        intervals.append([b-1,b])
        i+=1
    return intervals
    
def SturmIntervals(P):
    intervals=[]
    neg,x0,pos = numberOfRoots(P,-100,100)
    if neg!=x0:
        numeroRaices=neg-x0
        intervals=setRootsNegative(P,numeroRaices,intervals)
    if x0!=pos:
        numeroRaices = x0-pos
        intervals=setRootsPositive(P,numeroRaices,intervals)
    return intervals

In [9]:
P = np.poly1d([1,-9,24,-36]) #6
intervals = SturmIntervals(P)
print(intervals)
P = np.poly1d([1,-9.5,18,33.5,-97,18,36]) #-2,-1/2,1,2,3,6
intervals = SturmIntervals(P)
print(intervals)

[[5, 6]]
[[0, -1], [-2, -3], [0, 1], [1, 2], [2, 3], [5, 6]]


Compute the intervals of separation of the roots of the following polynomials:

$$P_1(x) = x^3-9x^2+24x-36$$
$$P_2(x) = x^6 - 9.5x^5 + 18x^4 + 33.5x^3 - 97x^2 + 18x +36$$